In [1]:
from pulp import LpProblem, LpMinimize, LpMaximize, LpVariable, lpDot, lpSum
from utilities import print_result

### Simple Linear Optimization

PuLP can solve the linear optimization problems in the following format.
$$
\begin{align}
\min & (3x - 5y) \\
\text {subject to} \; & y \leq x
\end{align}
$$

Here, $3x-5y$ is called the objective function. In this problem, we are trying to minimize this function, while making sure the *constraints* hold. In this problem, there is only one constraint, $y \leq x$.

In [2]:
# problem definition with name and 'sense'
problem = LpProblem('Simple Minimization', LpMinimize)

In [3]:
# Variables are defined with names, bounds, and categories
x = LpVariable('x', lowBound=0, cat='Continuous')
y = LpVariable('y', upBound=5, cat='Integer')

In [4]:
# the objective is added to the problem as a statement
problem += 3 * x - 5 * y

In [5]:
# the constraints are added as conditionals
problem += y <= x

In [6]:
problem.solve()

# problem status is 1 if optimized
print_result(problem)

Optimization status: 1
Final value of the objective: -10.0
Final values of the variables:
x = 5.0
y = 5.0


### Declaring a List of Variables

If there are multiple variables which have the same category, lower and upper bounds, it is possible to use a dictionary and list comprehensions to declare variables, objective, and constraints.

Consider a classic problem of maximixing utility with cost limitations.
$$
\max \sum_{i=1}^4 x_i u_i
$$
$$
\begin{align}
\text {subject to} \; \sum_{i=1}^4 x_i c_i &\leq 1 \\
0\leq x_i &\leq 20
\end{align}
$$

In [7]:
# A list of variables suffixes (x1, x2, x3, x4)
variable_names = [1, 2, 3, 4]

# Cost of each variables is defined in a dictionary
costs = {
    1: 0.05,
    2: 0.02,
    3: 0.07,
    4: 0.09
}

# Similarly utilities are defined in a dictionary
utilities = {
    1: 10,
    2: 8,
    3: 15,
    4: 20
}

In [8]:
# Variables are defined as a dictionary with comman bounds and category
# variables is a dictionary with keys being the elements of the variable_names list
# Values of this dict is the variable names (x_1, x_2...) which can be referred to define objectives and constraints
variables = LpVariable.dict('x', variable_names, lowBound=0, upBound=20, cat='Continuous')

In [9]:
problem = LpProblem('Maximize Utility', LpMaximize)

In [10]:
# Objective is defined as a dot product of the variables and corresponding utilities
problem += lpDot(variables.values(), utilities.values())

# Similarly, total cost is defined as the dot product of variables and costs
problem += lpDot(variables.values(), costs.values()) <= 1

In [11]:
problem.solve()

print_result(problem)

Optimization status: 1
Final value of the objective: 293.33333400000004
Final values of the variables:
x_1 = 0.0
x_2 = 20.0
x_3 = 0.0
x_4 = 6.6666667


### Integer Programs

Integer Programs: a linear program plus the additional constraints that some or all variables must be integer valued. Variables are called *binary* if they are allowed to take only 0 or 1 values.

### Logical Constraints

Logical constraints with two variables are rather easy.

**Constraint 1:** Either $x_1$ or $x_2$.
$$
x_1 + x_2 \leq 1
$$

Consider the above utility maximization problem. Let's say there are two items with known utilites and costs. Let's denote $x_i$ whether or not the item is bought. Let's try to maximize the utility with non-restricting total cost condition, allowing only one of the items to be selected.
$$
\max \sum_{i=1}^2 x_i u_i
$$
$$
\begin{align}
\sum_{i=1}^2 x_i c_i &\leq 100 \\
x_1 + x_2 &\leq 1
\end{align}
$$

In [12]:
variable_names = [1, 2]

# Costs are low, so even if both item are chosen, the cost limit (<= 100) is satisfied
costs = {
    1: 20,
    2: 10,
}

# Since the utility of 1 item is higher, that is expected to be chosen
utilities = {
    1: 10,
    2: 8
}

problem = LpProblem('Maximize Utility', LpMaximize)
variables = LpVariable.dict('x', variable_names, lowBound=0, upBound=1, cat='Integer')

# Objective
problem += lpDot(variables.values(), utilities.values())

# Cost constraint
problem += lpDot(variables.values(), costs.values()) <= 100

# OR constraint
problem += lpSum(variables.values()) <= 1

problem.solve()
print_result(problem)

Optimization status: 1
Final value of the objective: 10.0
Final values of the variables:
x_1 = 1.0
x_2 = 0.0


**Constraint 2:** If $x_1$ then $x_2$.
$$
x_1 \leq x_2
$$
Let's modify the above problem and add a third item. Let's impose the logical condition to be if item 1 is chosen then item 2 has to be chosen.
$$
\max \sum_{i=1}^3 x_i u_i
$$
$$
\begin{align}
\sum_{i=1}^3 x_i c_i &\leq 20 \\
x_1 &\leq x_2
\end{align}
$$

In [13]:
variable_names = [1, 2, 3]

# only one of the item can be chosen due to the cost limitation
costs = {
    1: 15,
    2: 15,
    3: 15,
}

# utility of 1 is highest, but choosing 1 will require choosing 2, but cost limitation restricts that
utilities = {
    1: 20,
    2: 5,
    3: 8,
}

problem = LpProblem('Maximize Utility', LpMaximize)
variables = LpVariable.dict('x', variable_names, lowBound=0, upBound=1, cat='Integer')

# Objective
problem += lpDot(variables.values(), utilities.values())

# Cost constraint
problem += lpDot(variables.values(), costs.values()) <= 20

# IF-THEN constraint
problem += variables.get(1) <= variables.get(2)

problem.solve()
print_result(problem)

Optimization status: 1
Final value of the objective: 8.0
Final values of the variables:
x_1 = 0.0
x_2 = 0.0
x_3 = 1.0


Similarly, at least one of $x_1$ and $x_2$ can be imposed with the following condition.
$$
x_1 + x_2 \geq 1
$$
Both, $x_1$ and $x_2$ can be imposed with the following condition.
$$
x_1 + x_2 \geq 2
$$

### Logical Constraints with Non-Binary Variables
Continuous variables are harder to model. Consider the following condition $x \leq 5$ or $x \geq 8$. We introduce a number *M* which is much larger than any variable in the problem and we introduce a binary variable $w$ and rewrite the equations as:
$$
\begin{align}
x &\leq 5 + M(1-w) \\
x &\geq 8 - Mw
\end{align}
$$
Adding a very large number to RHS of a less than or equal to condition automatially satisfies that. Similarly, subtracting a very large number from RHS of a greater than or equal to condition automatically satisfies that. Binary variable $w$ makes sure only one of the equations is satisfied and the other is imposed.

Consider the following problem.
$$
\min x + y
$$
$$
\begin{align}
0 &\leq x \\
0 &\leq y
\end{align}
$$
$$
x \geq 10 \; or \; y \geq 5
$$
We formulate the problem as follows:
$$
\begin{align}
\min x &+ y\\
0 &\leq x \\
0 &\leq y \\
x &\geq 10 - M(1-w)\\
y &\geq 5 - Mw
\end{align}
$$

In [14]:
problem = LpProblem('Minimize', LpMinimize)

x = LpVariable('x', lowBound=0, cat='Continuous')
y = LpVariable('y', lowBound=0, cat='Continuous')
w = LpVariable('w', lowBound=0, upBound=1, cat='Integer')

M = 10

problem += x + y

problem += x >= 10 - M * (1 - w)
problem += y >= 5 - M * w

problem.solve()
print_result(problem)

Optimization status: 1
Final value of the objective: 5.0
Final values of the variables:
w = 0.0
x = 0.0
y = 5.0


Since at least one of the conditions has to be satisfied, and condition on y puts a lower limit on the variable and we are trying to minimize the sum, second condition is satisfied and not the first one.

#### Modeling IF-THEN type constraints
If-Then type constraints can be first converted to or conditions with the followint equivalence:
$$
p \implies q \equiv \lnot p \vee q
$$
For example, if $x + y \leq 10$ then $x \geq 5$ condition can be rewritten as $x + y \geq 10$ or $x \geq 5$. When negating a statement, inequality switches. Then the or condition can be modeled as described above.
Let's model the following problem in PuLP.
$$
\begin{align}
\min (x &+ y) \\
0 &\leq x, y \\
x + y \leq 10 &\implies x \geq 5
\end{align}
$$
We can rewrite the problem as:
$$
\begin{align}
\min (x &+ y) \\
0 &\leq x, y \\
x + y &\geq 10 - M(1 - w)\\
x &\geq 5 - Mw
\end{align}
$$

In [15]:
problem = LpProblem('Minimize', LpMinimize)

x = LpVariable('x', lowBound=0, cat='Continuous')
y = LpVariable('y', lowBound=0, cat='Continuous')
w = LpVariable('w', lowBound=0, upBound=1, cat='Integer')

M = 100

problem += x + y

problem += x + y >= 10 - M * (1 - w)
problem += x >= 5 - M * w

problem.solve()
print_result(problem)

Optimization status: 1
Final value of the objective: 5.0
Final values of the variables:
w = 0.0
x = 5.0
y = 0.0


Since, we are trying to minimize the summation of the two variables, they try to approach their lower limit, 0. The if condition enforces x be greater than or equal to 5.

### Fixed Charge Problems
This is a special type of a more general family of problems, piecewise defined function. In this section we will model the function of the following form.
$$
f(x)=   \left\{
\begin{array}{ll}
      0    & x = 0 \\
      ax+b & x > 0 \\
\end{array} 
\right.
$$

Consider the following profit maximization problem. Each product requires using some limited resource and there is a fixed charge of producing that product. We want to optimize the number of each products to be produced that maximizes the profit.

|&nbsp;      |Product 1|Product 2|Product 3|Available|
|------------|---------|---------|---------|---------|
|Resource 1  |    3    |    4    |    5    |   100   |
|Resource 2  |    1    |    2    |    3    |    30   |
|Resource 3  |   10    |    5    |    2    |   204   |
|Profit      |   40    |   30    |   20    |         |
|Fixed Charge|  500    |  400    |  300    |         |

Let $x_i$ denote the number of product $i$ to be produced, $p_i$ the corresponding profit, $c_i$ the corresponding fixed charge. We can then define the profit to be gained from the product $i$ as follows:
$$
f_i(x_i)=   \left\{
\begin{array}{ll}
      0              & x_i = 0 \\
      p_i x_i - c_i  & x_i > 0 \\
\end{array} 
\right.
$$
Then, we want to maximize, the total profit:
$$
\max \sum_{i=1}^3 f_i(x_i)
$$
We introduce the binary variables $w_i$ such that $x_i \leq Mw_i$. If $w_i$ is $0$ then $x_i$ is enforced to be $0$. For any other value of $x_i$, $w_i$ must be $1$. This allows us to write the piecewise defined profit function as follows:
$$
f_i(x_i) = p_i x_i - w_i c_i
$$

In [16]:
problem = LpProblem("Fixed Charge Cost Maximization", LpMaximize)

profit = {1: 40, 2: 30, 3: 20}
fixed_charge = {1: 500, 2: 400, 3: 300}

resources = {
    1: [3, 4, 5],
    2: [1, 2, 3],
    3: [10, 5, 2]
}
availability = {1: 100, 2: 30, 3: 204}

products = [1, 2, 3]

product_variables = LpVariable.dict('x', products, lowBound=0, cat='Integer')
binary_variables = LpVariable.dict('w', products, lowBound=0, upBound=1, cat='Integer')

M = 1000

# impose the condition x_i <= M w_i
for i in products:
    problem += product_variables[i] <= M * binary_variables[i]

    
# resource limitation
for i in resources:
    problem += lpDot(resources[i], product_variables.values()) <= availability[i]


# define objective as sum(p_i x_i) - sum(c_i w_i)
problem += lpDot(profit.values(), product_variables.values()) - lpDot(fixed_charge.values(), binary_variables.values())

problem.solve()
print_result(problem)

Optimization status: 1
Final value of the objective: 300.0
Final values of the variables:
w_1 = 1.0
w_2 = 0.0
w_3 = 0.0
x_1 = 20.0
x_2 = 0.0
x_3 = 0.0


### Piecewise Linear Functions
We can generalize the idea introduced above to model piecewise defined functions. Assume cost is defined by the following function, and x is integer valued.
$$
f(x)=   \left\{
\begin{array}{ll}
      4x       & 0 \leq x \leq 3 \\
      3x + 3   & 4 \leq x \leq 7 \\
      2x + 10  & 8 \leq x \leq 9 \\
\end{array} 
\right.
$$
We can define the following integer and binary variables.
$$
w_1 =   \left\{
\begin{array}{ll}
      1       & 0 \leq x \leq 3 \\
      0       & \text{otherwise} \\
\end{array} 
\right.
$$
$$
w_2 =   \left\{
\begin{array}{ll}
      1       & 4 \leq x \leq 7 \\
      0       & \text{otherwise} \\
\end{array} 
\right.
$$
$$
w_3 =   \left\{
\begin{array}{ll}
      1       & 8 \leq x \leq 9 \\
      0       & \text{otherwise} \\
\end{array} 
\right.
$$
$$
x_1 =   \left\{
\begin{array}{ll}
      x       & 0 \leq x \leq 3 \\
      0       & \text{otherwise} \\
\end{array} 
\right.
$$
$$
x_2 =   \left\{
\begin{array}{ll}
      x       & 4 \leq x \leq 7 \\
      0       & \text{otherwise} \\
\end{array} 
\right.
$$
$$
x_3 =   \left\{
\begin{array}{ll}
      x       & 8 \leq x \leq 9 \\
      0       & \text{otherwise} \\
\end{array} 
\right.
$$
This is summarized in the following table:

|&nbsp;|$0 \leq x \leq 3$|$4 \leq x \leq 7$|$8 \leq x \leq 9$|
|------|-----------------|-----------------|-----------------|
|$w_1$ |        1        |        0        |        0        |
|$w_2$ |        0        |        1        |        0        |
|$w_3$ |        0        |        0        |        1        |
|$x_1$ |       $x$       |        0        |        0        |
|$x_2$ |        0        |       $x$       |        0        |
|$x_3$ |        0        |        0        |       $x$       |


Then we can define the piecewise defined functions as follows:
$$
f(x) = 4x_1 + (3x_2 + 3w_2) + (2x_3 + 10w_3)
$$
We can set the following constraints.
$$
\begin{align}
0 & \leq x_1 \leq 3w_1 \\
4w_2 & \leq x_2 \leq 7w_2 \\
8w_3 & \leq x_3 \leq 9w_3
\end{align}
$$
$$
w_1 + w_2 + w_3 = 1
$$
$$
x_1 + x_2 + x_3 = x
$$
Let's assume above function is a revenue of a process and the cost of the same process is defined by $g(x) = 3x$. We can find the value of $x$ that maximizes the profit in the next code block.

In [24]:
problem = LpProblem("Maximize profit with piecewise cost", LpMaximize)
x = LpVariable('x', lowBound=0, upBound=9, cat='Integer')
subscripts = [1, 2, 3]
x_i = LpVariable.dict('x', subscripts, lowBound=0, upBound=9, cat='Integer')
w_i = LpVariable.dict('w', subscripts, lowBound=0, upBound=1, cat='Integer')

problem += (4 * x_i[1] + 3 * x_i[2] + 3 * w_i[2] + 2 * x_i[3] + 10 * w_i[3]) - 3*x
problem += 0 <= x_i[1]
problem += x_i[1] <= 3 * w_i[1]
problem += 4 * w_i[2] <= x_i[2]
problem += x_i[2] <= 7 * w_i[2]
problem += 8 * w_i[3] <= x_i[3]
problem += x_i[3] <= 9 * w_i[3]
problem += lpSum(w_i.values()) == 1
problem += lpSum(x_i.values()) == x

problem.solve()
print_result(problem)

Optimization status: 1
Final value of the objective: 3.0
Final values of the variables:
w_1 = 1.0
w_2 = 0.0
w_3 = 0.0
x = 3.0
x_1 = 3.0
x_2 = 0.0
x_3 = 0.0
